In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 1. load data

In [ ]:
test = pd.read_csv("../input/nlp-getting-started/test.csv")
train = pd.read_csv("../input/nlp-getting-started/train.csv")
submission = pd.read_csv("../input/nlp-getting-started/sample_submission.csv")


What am I predicting?

  - You are predicting whether a given tweet is about a real disaster or not. If so, predict a 1. If not, predict a 0.

Files
 - train.csv - the training set
 - test.csv - the test set
 - sample_submission.csv - a sample submission file in the correct format
 
Columns
 - id - a unique identifier for each tweet
 - text - the text of the tweet
 - location - the location the tweet was sent from (may be blank)
 - keyword - a particular keyword from the tweet (may be blank)
 - target - in train.csv only, this denotes whether a tweet is about a real disaster (1) or not (0)

In [ ]:
train.sample(10)

In [ ]:
train.info()

# 2. visualization

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt


실제 재난 트윗와 재난 트윗이 아닌 것의 양

In [ ]:
sns.countplot(x="target", data=train)
plt.title("Disaster(1) or Not Disaster(0)")
plt.show()

**Disaster**

In [ ]:
from wordcloud import WordCloud

In [ ]:
text1 = dict(train[train.target==1].keyword.value_counts())
wordcloud = WordCloud(width=800, height=400,background_color="white").generate_from_frequencies(text1)
plt.figure(figsize=[14,8])
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

**not disaster**

In [ ]:
text2 = dict(train[train.target==0].keyword.value_counts())
wordcloud = WordCloud(width=800, height=400,background_color="white").generate_from_frequencies(text2)
plt.figure(figsize=[14,8])
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

# 3.DATA PROCESSING

# 특수문자 제거

In [ ]:
from tqdm import tqdm
import re
#tqdm? : progress bar 나타내어 진행상황 알려줌. 없어도 상관 X
text_list = list(train['text'])

clear_text_list = [] #특수문자를 제거한 text를 입력

for i in tqdm(range(len(text_list))):
    clear_text = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]','',text_list[i])
    #re.sub 이용하여 공백으로 치환
    clear_text_list.append(clear_text.lower())
    #lower 이용해 소문자로 변경
    
train['clear_text'] = clear_text_list
#column 추가
train

test데이터도 똑같이 처리

In [ ]:
from tqdm import tqdm
import re
#tqdm? : progress bar 나타내어 진행상황 알려줌. 없어도 상관 X
text_list = list(test['text'])

clear_text_list = [] #특수문자를 제거한 text를 입력

for i in tqdm(range(len(text_list))):
    clear_text = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]','',text_list[i])
    #re.sub 이용하여 공백으로 치환
    clear_text_list.append(clear_text.lower())
    #lower 이용해 소문자로 변경
    
test['clear_text'] = clear_text_list
#column 추가
test

# 토큰화 / 불용어 처리 / 어간 추출

tokenization : 모든 구문을 단어, 문자, 하위단어와 같은 더 작은 섹션으로 분할하는 작업. 여기서는 문장을 단어단위로 분할

stopword 제거 : I, my, me, over, 조사, 접미사 같은 단어들은 문장에서는 자주 등장하지만 실제 의미 분석을 하는데는 거의 기여하는 바가 없는 경우 있으므로 제거

stemming : 접사를 제거하여 어간을 추출하는 과정 예를 들어 'automate’, ‘automatic’, ‘automation’과 같은 단어를 automat으로 바꿔주는 작업

In [ ]:
import nltk
#자연어 처리 패키지 Natural Language Toolkit
nltk.download("stopwords")
nltk.download("punkt")
#토큰화

In [ ]:
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
#from nltk.stem import LancasterStemmer
X_train = []

print(stopwords.words('english'))

stemmer = PorterStemmer()
#l=LancasterStemmer()
stop_words = set(stopwords.words('english'))


clear_text_list = list(train['clear_text'])

for i in tqdm(range(len(clear_text_list))):
    temp = word_tokenize(clear_text_list[i])
    # 토큰화 : 문장을 단어단위로 구분
    temp = [word for word in temp if word not in stop_words]
    #불용어 제거
    temp = [stemmer.stem(word) for word in temp]
    # 어간추출(stemming)
    temp = [word for word in temp if len(word) > 1]
    #단어 길이
    X_train.append(temp)
print(train['text'][0])
print(X_train[0])

test데이터 똑같이 처리

In [ ]:
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
X_test = []
stemmer = PorterStemmer()
stop_words = set(stopwords.words('english'))
clear_text_list = list(test['clear_text'])

for i in tqdm(range(len(clear_text_list))):
    temp = word_tokenize(clear_text_list[i])
    # word_tokenize : 문장을 단어단위로 구분
    temp = [word for word in temp if word not in stop_words]
    #불용어 제거
    temp = [stemmer.stem(word) for word in temp]
    # 어간추출(stemming)
    
    temp = [word for word in temp if len(word) > 1]
    X_test.append(temp)
print(test['text'][0])
X_test[0]

# 단어를 정수로 인코딩

Why? 컴퓨터는 텍스트보다 숫자를 더 잘 처리한다.

이 과정은 우리가 흔히 아는 원-핫 인코딩에도 이용된다.

    1. 정수인코딩 수행
    2. 부여된 고유정수를 인덱스로 간주하여 해당 위치에 1을 부여한다.

이 중 1번 과정만 실행한다.


In [ ]:
words = []

for i in tqdm(range(len(X_train))):
    for j in range(len(X_train[i])):
        words.append(X_train[i][j])
len(list(set(words)))

In [ ]:
from keras.preprocessing.text import Tokenizer
max_words = 19526
tokenizer = Tokenizer(num_words = max_words)
tokenizer.fit_on_texts(X_train)
#단어집합 생성
X_train_vec = tokenizer.texts_to_sequences(X_train)
X_test_vec = tokenizer.texts_to_sequences(X_test)
#정수 시퀀스로 변환

In [ ]:
print(X_train_vec[:1])

print(X_test_vec[:1])

# 토큰화된 트윗의 길이 맞춰주기(padding)

In [ ]:
import matplotlib.pyplot as plt

print("최대 길이 : ", max(len(i) for i in X_train))
print("평균 길이 : ", sum(map(len,X_train))/len(X_train))
plt.hist([len(s) for s in X_train], bins=50)
plt.xlabel('length of Data')
plt.ylabel('number of Data')
plt.show

In [ ]:
from keras.preprocessing.sequence import pad_sequences
max_len = 23

X_train_vec = pad_sequences(X_train_vec, maxlen=max_len)
X_test_vec = pad_sequences(X_test_vec, maxlen=max_len)

In [ ]:
print(X_train_vec[:1])

print(X_test_vec[:1])

# target값 One-Hotecoding 

In [ ]:
from keras.utils import np_utils
import numpy as np

y_train = []

for i in range(len(train['target'])):
    if train['target'].iloc[i] ==1:
        y_train.append([0,1])
    elif train['target'].iloc[i] ==0:
        y_train.append([1,0])
        
y_train = np.array(y_train)

y_train

# 4.**modeling**

Embedding?(https://casa-de-feel.tistory.com/28)

단어를 연속적인 벡터로 매핑 하는 것

Ex) 강아지 = [0.2 1.8 1.1 -2.1 1.1 2.8 ... 중략 ...]

자연어일 때 불가능했던 유사도 계산이 임베딩 덕분에 가능

원핫인코딩의 한계

    1. 카테고리 하나 추가될 때마다 벡터공간 한개 추가. 공간적 낭비
    
    ex) 10000개의 단어가 있으면  10000차원의 벡터가 필요
    
    2. 모든 단어가 0,1로 동일하게 되면 단어간 유사도 측정 불가

LSTM(Lont Short-Term Memory)

https://pleasurehwang.tistory.com/19

말 그대로 순서가 있는 data를 시퀀스라고 함. 예를 들어 Text 에는 문맥이라는 순서가 있으므로 시퀀스 데이터 라고 할 수 있다.

그리고 RNN(순환 신경망)은 연속형 데이터를 처리하기 위해 고안된 신경망 구조이므로 RNN을 사용하고자 하였다.

LSTM은 가장 기본적인 RNN인 vanilla RNN을 발전시킨 것으로 장기 메모리와 단기 메모리에 들어갈 정보를 나눠 학습한다.

기존의 RNN은 먼 과거의 일로부터 학습하는 것이 산술적으로 거의 불가능했지만, LSTM은 수백만 단위 시간 전의 사건으로부터도 학습할 수 있다.


In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, Dropout

model = Sequential()
model.add(Embedding(max_words, 100))
#차원이 100인 벡터를 만든다.
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(Dense(2, activation = 'sigmoid'))

model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

history = model.fit(X_train_vec, y_train, epochs=1,batch_size = 32, validation_split=0.1)

# 5.SUBMISSION

In [ ]:
submission.head()

In [ ]:
predict = model.predict(X_test_vec)
predict_labels = np.argmax(predict, axis=1)

for i in range(len(predict_labels)):
    predict_labels[i] = predict_labels[i]
    
ids = list(test['id'])
    
submission_dic = {"id":ids, "target":predict_labels}
submission_df = pd.DataFrame(submission_dic)
submission_df.to_csv("kaggle_01.csv", index = False)